In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import pandas as pd
import math
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import os
import itertools
import numpy as np

In [2]:
class TextValueDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, column):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.column = column

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        text = row['generated_text']
        label = (row[self.column] - 1)/4 # Convert value to 0-1
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

In [3]:
# Load dataset
def load_dataset(file_path):
    if '.tsv' in file_path:
        df = pd.read_csv(file_path, sep='\t')
    else:
        df = pd.read_csv(file_path)
    return df

# Create DataLoader
def create_dataloader(df, tokenizer, max_length, batch_size, column):
    dataset = TextValueDataset(df, tokenizer, max_length, column)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
def evaluate(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []

    eval_loss = 0
    eval_loop = tqdm(dataloader, desc="Evaluating", leave=False)
    with torch.no_grad():
        for batch in eval_loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # print(input_ids, attention_mask, labels)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            
            loss = outputs.loss
            eval_loss += loss.item()
            eval_loop.set_postfix(loss=loss.item())
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    # accuracy = accuracy_score(true_labels, predictions)
    # precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
    
    mae = mean_absolute_error(true_labels, predictions)
    mse = mean_squared_error(true_labels, predictions)
    return eval_loss, np.sqrt(mse), mse, mae

In [5]:
data_dict = { 
                'model' : [],
                 'dimension' : [],
                 'RMSE' : [],
                 'MSE' : [],
                 'MAE' : []
            }

In [6]:
def get_performance(tokname, modeldir):
    max_length = 256
    batch_size = 16
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-small')
    tokenizer = AutoTokenizer.from_pretrained(tokname)
    test_df = load_dataset('../data/crowd-enVent-test.tsv')
    for model_name in os.listdir(modeldir):
        # model_name='./models/google-t5/suddenness_google-t5/'
        if '.ipy' in model_name:
            continue
        column = str.join('_', model_name.split('_')[:-1])
        print(column)
        test_loader = create_dataloader(test_df, tokenizer, max_length, batch_size, column)
        model = AutoModelForSequenceClassification.from_pretrained(os.path.join(modeldir, model_name))
        model = model.to(device)
        _, test_rmse, test_mse, test_mae = evaluate(model, test_loader, device)
        print(f'Model Name: {model_name}, Test RMSE: {test_rmse:.4f}, Test MSE: {test_mse:.4f}, Test MAE: {test_mae:.4f}')
        data_dict['model'].append(model_name.split('_')[-1])
        data_dict['dimension'].append(column)
        data_dict['RMSE'].append(test_rmse)
        data_dict['MSE'].append(test_mse)
        data_dict['MAE'].append(test_mae)
        # Cleanup
        model.to('cpu')
        del model
        torch.cuda.empty_cache()
    
    return data_dict

In [7]:
mod_tok_dict = {
    'bert-large-uncased' : 'bert-large-uncased',
    'dialogpt' : 'microsoft/DialogRPT-updown',
    'distilbert-base-uncased' : 'distilbert-base-uncased',
    'google-t5' : 'google-t5/t5-small',
    'roberta-base' : 'roberta-base'
}

for key in mod_tok_dict:
    modeldir = os.path.join('./models', key)
    data_dict = get_performance(mod_tok_dict[key], modeldir)

/home/adebnath/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


goal_relevance


Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:09<00:00,  3.75it/s, loss=0.0945]


Model Name: goal_relevance_bert-large-uncased, Test RMSE: 0.6449, Test MSE: 0.4159, Test MAE: 0.5236
chance_responsblt


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:08<00:00,  4.08it/s, loss=0.122]


...

Model Name: familiarity_roberta-base, Test RMSE: 0.5424, Test MSE: 0.2942, Test MAE: 0.3926
predict_conseq


Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:02<00:00, 11.49it/s, loss=0.0741]


Model Name: predict_conseq_roberta-base, Test RMSE: 0.6203, Test MSE: 0.3847, Test MAE: 0.5046
not_consider


Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:03<00:00, 11.18it/s, loss=0.0943]


Model Name: not_consider_roberta-base, Test RMSE: 0.6002, Test MSE: 0.3602, Test MAE: 0.4426


In [11]:
df = pd.DataFrame(data_dict)
df

,model,dimension,RMSE,MSE,MAE
0,bert-large-uncased,goal_relevance,0.644869,0.415856,0.523611
1,bert-large-uncased,chance_responsblt,0.533160,0.284259,0.363889
2,bert-large-uncased,chance_control,0.512348,0.262500,0.333333
3,bert-large-uncased,accept_conseq,0.615295,0.378588,0.501389
4,bert-large-uncased,standards,0.552980,0.305787,0.383333
...,...,...,...,...,...
95,roberta-base,standards,0.552980,0.305787,0.383333
96,roberta-base,self_responsblt,0.586401,0.343866,0.418056
97,roberta-base,familiarity,0.542414,0.294213,0.392593
98,roberta-base,predict_conseq,0.620260,0.384722,0.504630


In [17]:
df[df['model'] == 'bert-large-uncased'].describe()

,RMSE,MSE,MAE
count,20.000000,20.000000,20.000000
mean,0.598978,0.365822,0.454537
std,0.086130,0.101718,0.112351
min,0.389533,0.151736,0.199537
25%,0.541181,0.292882,0.378472
50%,0.594705,0.353704,0.446296
75%,0.662285,0.438628,0.530903
max,0.752465,0.566204,0.676852


In [18]:
df[df['model'] == 'roberta-base'].describe()

,RMSE,MSE,MAE
count,20.000000,20.000000,20.000000
mean,0.598978,0.365822,0.454537
std,0.086130,0.101718,0.112351
min,0.389533,0.151736,0.199537
25%,0.541181,0.292882,0.378472
50%,0.594705,0.353704,0.446296
75%,0.662285,0.438628,0.530903
max,0.752465,0.566204,0.676852


In [19]:
df[df['model'] == 'distilbert-base-uncased'].describe()

,RMSE,MSE,MAE
count,20.000000,20.000000,20.000000
mean,0.598978,0.365822,0.454537
std,0.086130,0.101718,0.112351
min,0.389533,0.151736,0.199537
25%,0.541181,0.292882,0.378472
50%,0.594705,0.353704,0.446296
75%,0.662285,0.438628,0.530903
max,0.752465,0.566204,0.676852


In [20]:
df[df['model'] == 'google-t5'].describe()

,RMSE,MSE,MAE
count,20.000000,20.000000,20.000000
mean,0.598978,0.365822,0.454537
std,0.086130,0.101718,0.112351
min,0.389533,0.151736,0.199537
25%,0.541181,0.292882,0.378472
50%,0.594705,0.353704,0.446296
75%,0.662285,0.438628,0.530903
max,0.752465,0.566204,0.676852
